In [3]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from tqdm.notebook import tqdm

import os
import sys
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.nn as nn
import torch
import torch.optim as optim

sys.path.append("../src")

#here you can find utils that are used in each notebook
from utils import set_seed, EarlyStoppingR2, train_CFG

# Data Processing

In [4]:
region = 777

regions_mapper = {
    777 : "Moscow",
    812 : "Saint-Petersburg",
    287 : "Kazan",
    473 : "Sochi"
}

pipeline = "efficient_delete_channels" #("basic", "efficient_all_channels", "efficient_delete_channels")

# path to graph with images
path_to_graph = f"/home/jupyter/datasphere/s3/s3-sirius/sirius_2024_participants/twwist/graph_with_cv_full_and_images/graph_with_cv_full_and_images_{region}_with_image_features.pickle"
path_to_save = f"../data/image_embeddings/image_embeddings_{pipeline}_{region}.pickle"

#loading with torch
graph = torch.load(path_to_graph)

ys = graph.y
mask_to_null_ys = (ys != -1)

xs = graph.imgs[mask_to_null_ys]
ys = ys[mask_to_null_ys].tolist()

In [ ]:
def images_features(dataset):
    a = []
    data = {}
    for i in dataset.imgs:
        a.append(i)
    m1 = np.zeros_like(dataset.imgs[199][0]) 
    m1[26:, 26:] = 1      

    m2 = np.zeros_like(dataset.imgs[199][0]) 
    m2[26:, :27] = 1    

    m3 = np.zeros_like(dataset.imgs[199][0])  
    m3[:27:, :27] = 1   

    m4 = np.zeros_like(dataset.imgs[199][0]) #1
    m4[:27:, 26:] = 1 
    
    
    for i in range(len(a)):
        for j in range(12):
            if i not in data:
                data[i] = [(dataset.imgs[i][j] * m1 > 0).sum().item()]
            else:
                data[i] += [(dataset.imgs[i][j] * m1 > 0).sum().item()]
            data[i] += [(dataset.imgs[i][j] * m2 > 0).sum().item()]
            data[i] += [(dataset.imgs[i][j] * m3 > 0).sum().item()]
            data[i] += [(dataset.imgs[i][j] * m4 > 0).sum().item()]
    
    for i in range(len(a)):
        for j in range(12):
            if i not in data:
                data[i] = [(dataset.imgs[i][j] > 0).sum().item()]
            else:
                data[i] += [(dataset.imgs[i][j] > 0).sum().item()]
    
    df = pd.DataFrame.from_dict(data, orient="index")
    res = df.apply(pd.Series.explode)
    
    res.columns = ['num', 'm10','m20','m3','m40','m11', 'm21', 'm31', 'm41',
                   'm12','m22','m32','m42','m13','m23','m33','m43','m14','m24',
                   'm34','m44','m15','m25','m35', 'm16','m26','m36','m46','m17',
                   'm27','m37','m47','m18','m28','m38','m48','m19','m29','m39',
                   'm49','m110','m210','m310','m410','m111','m211','m311','m411',
                   'summ0', 'summ1', 'summ2','summ3','summ4','summ5','summ6',
                   'summ7','summ8','summ9','summ10','summ11']
    
    return torch.FloatTensor(res.values) / (52 * 52)

resulted_features = images_features(graph)


In [3]:
if "efficient" in pipeline:
    %pip install timm -q
    import timm


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
class RasterDataset(torch.utils.data.Dataset):
    def __init__(self, images, y):

        self.images = images
        self.y = y

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        img = self.images[idx]
        y = self.y[idx]
        
        return img, y
    

class TimmRasterDataset(torch.utils.data.Dataset):
    def __init__(self, images, y, timm_transform):

        self.images = images.to(torch.float32)
        self.y = y
        self.transform = transforms.Resize((224, 224), antialias=True)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = self.transform(self.images[idx])
        
        return image

In [9]:
if pipeline == "basic":
    full_dataset = RasterDataset(xs, ys)
else:
    
    if pipeline == "efficient_delete_channels":
        # throwing out some trash channels
        graph.imgs = graph.imgs[:, [0, 3, 4, 5, 6, 7, 8, 9, 10]]
    
    full_dataset = TimmRasterDataset(graph.imgs, graph.y, transforms)

In [10]:
if not os.path.exists("../chkps/image_checkpoints"):
    !mkdir ../chkps/image_checkpoints

class cfg:
    num_epochs = 100
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
cfg = cfg()

# Timm model


In [11]:
feature_extractor = timm.create_model('efficientnet_b0.ra_in1k',
        pretrained=True,
        features_only=True,
        in_chans=1,
        out_indices=[2]
).to(cfg.device)

model = timm.create_model('efficientnet_b0.ra_in1k',
        pretrained=True,
        features_only=False,
        in_chans=12,
).to(cfg.device)

dataloader_all_images = torch.utils.data.DataLoader(
    full_dataset, 
    batch_size=1, 
    shuffle=False, 
    num_workers=4, 
    pin_memory=True
)

use_feature_exctractor = (False if pipeline == "efficient_all_channels" else True) 

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


In [12]:
model.eval()

t = nn.AvgPool2d(kernel_size=(28, 28))

image_embeds = []

with torch.no_grad():
    for images in tqdm(dataloader_all_images):
        if use_feature_exctractor:
            images = images.flatten(end_dim=1).unsqueeze(1)
            output = feature_extractor(images.to(cfg.device))[0]
            output = t(output).flatten(start_dim=1)
            output = torch.cat([x for x in output], dim=0)
        else:
            output = model.forward_features(images.to(cfg.device))
            output = model.forward_head(output, pre_logits=True)
        image_embeds.append(output)
        del output
        torch.cuda.empty_cache()

  0%|          | 0/11557 [00:00<?, ?it/s]

In [13]:
image_embeds = torch.cat(image_embeds, dim=0).view(len(image_embeds), -1)
torch.save(image_embeds.cpu(), path_to_save)

# Base Model

In [123]:
class RasterCNN(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels, 64, 5, padding=(0, 0))
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 5, padding=(0, 0))
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=(0, 0))
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=(0, 0))
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.AvgPool2d(kernel_size=40, stride=1, padding=0)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x, return_hidden=False):
        
        x = F.relu(self.conv1(x))
        x = self.bn1(x)


        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)


        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        
        if return_hidden: hidden_state = x

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x if not return_hidden else (x, hidden_state)

In [ ]:
train_set, val_set = torch.utils.data.random_split(full_dataset, [0.9, 0.1])

batch_size = 64

dataloader_train = torch.utils.data.DataLoader(
    train_set, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=4, 
    pin_memory=True
)
dataloader_val = torch.utils.data.DataLoader(
    val_set, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=4, 
    pin_memory=True
)

In [ ]:
seed_everything(42)

model = RasterCNN(12).to(cfg.device)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=7e-4)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer, 
    base_lr=4e-5, 
    max_lr=2e-4, 
    cycle_momentum = False, 
    step_size_up=20,
)


best_metric = float('inf')
best_epoch = -1

for epoch in range(cfg.num_epochs):

    loss_i_train = []
    r2_i_train = []
    
    model.train()
    
    for images, targets in tqdm(dataloader_train, desc='Training'):

        images = images.to(cfg.device)
        targets = targets.to(cfg.device)

        optimizer.zero_grad()

        outputs = model(images.float()).squeeze()
        loss = criterion(outputs, targets.float())
        loss.backward()
        optimizer.step()

        loss_i_train.append(loss.item() ** 0.5)
        r2_i_train.append(
            r2_score(targets.cpu(), outputs.detach().cpu())
        )

    print(f"Epoch: {epoch}, Train rmse: {np.mean(loss_i_train)} r2: {np.mean(r2_i_train)}")

    loss_i_val = []
    r2_i_val = []
    
    model.eval()

    with torch.no_grad():
        for images, targets in dataloader_val:

            images = images.to(cfg.device)
            targets = targets.to(cfg.device)

            outputs = model(images.float()).squeeze()
            
            loss_i_val.append(loss.item() ** 0.5)
            r2_i_val.append(
                r2_score(targets.cpu(), outputs.detach().cpu())
            )

    print(f"Validation rmse: [{np.mean(loss_i_val)}], r2: [{np.mean(r2_i_val)}]")

    if np.mean(loss_i_val) - best_metric < -1e6 / 2:
        best_metric = np.mean(loss_i_val)
        best_epoch = epoch
        folder_path = "../chkps/image_embeddings/"
        
        if epoch != 0:

            torch.save(model, f'{folder_path}/weights_{epoch}_{round(best_metric, 5)}.pt')


print('Finished Training')

In [139]:
all_images_dataset = RasterDataset(graph.imgs, graph.y)

dataloader_all_images = torch.utils.data.DataLoader(
    all_images_dataset, 
    batch_size=1, 
    shuffle=False, 
    num_workers=4, 
    pin_memory=True
)

hidden_states = []

best_model = torch.load(f'{folder_path}/weights_{best_epoch}_{round(best_metric, 5)}.pt')

In [140]:
best_model.eval()
with torch.no_grad():
    for image, _ in tqdm(dataloader_all_images):
        _, h = best_model(image.to(cfg.device).float(), return_hidden=True)
        hidden_states.append(h.squeeze(0))

  0%|          | 0/11557 [00:00<?, ?it/s]

In [141]:
hidden_states = torch.cat(hidden_states, dim=0).view(-1, hidden_states[0].shape[0])
hidden_states = hidden_states.to(torch.device('cpu'))
torch.save(graph, path_to_save)

In [16]:
%%python image_preprocessing.py

UsageError: %%python is a cell magic, but the cell body is empty.


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from tqdm.notebook import tqdm

import os
import sys
import timm
import torch
import argparse 
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

sys.path.append("../src")

#here you can find utils that are used in each notebook
from utils import set_seed, EarlyStoppingR2, train_CFG
from datasets import RasterDataset, TimmRasterDataset
from models import RasterCNN

def get_images_features(dataset):
    a = []
    data = {}
    for i in dataset.imgs:
        a.append(i)
    m1 = np.zeros_like(dataset.imgs[199][0]) 
    m1[26:, 26:] = 1      

    m2 = np.zeros_like(dataset.imgs[199][0]) 
    m2[26:, :27] = 1    

    m3 = np.zeros_like(dataset.imgs[199][0])  
    m3[:27:, :27] = 1   

    m4 = np.zeros_like(dataset.imgs[199][0]) #1
    m4[:27:, 26:] = 1 
    
    
    for i in range(len(a)):
        for j in range(12):
            if i not in data:
                data[i] = [(dataset.imgs[i][j] * m1 > 0).sum().item()]
            else:
                data[i] += [(dataset.imgs[i][j] * m1 > 0).sum().item()]
            data[i] += [(dataset.imgs[i][j] * m2 > 0).sum().item()]
            data[i] += [(dataset.imgs[i][j] * m3 > 0).sum().item()]
            data[i] += [(dataset.imgs[i][j] * m4 > 0).sum().item()]
    
    for i in range(len(a)):
        for j in range(12):
            if i not in data:
                data[i] = [(dataset.imgs[i][j] > 0).sum().item()]
            else:
                data[i] += [(dataset.imgs[i][j] > 0).sum().item()]
    
    df = pd.DataFrame.from_dict(data, orient="index")
    res = df.apply(pd.Series.explode)
    
    res.columns = ['num', 'm10','m20','m3','m40','m11', 'm21', 'm31', 'm41',
                   'm12','m22','m32','m42','m13','m23','m33','m43','m14','m24',
                   'm34','m44','m15','m25','m35', 'm16','m26','m36','m46','m17',
                   'm27','m37','m47','m18','m28','m38','m48','m19','m29','m39',
                   'm49','m110','m210','m310','m410','m111','m211','m311','m411',
                   'summ0', 'summ1', 'summ2','summ3','summ4','summ5','summ6',
                   'summ7','summ8','summ9','summ10','summ11']
    return torch.FloatTensor(res.values) / (52 * 52)

class cfg:
    num_epochs = 100
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg = cfg()


pipeline = "efficient_all_channels"
region = 777
path_to_graph = "/home/jupyter/datasphere/s3/s3sirius/sirius_2024_participants/twwist/graph_with_cv_full_and_images/"

path_to_save = "../data/image_embeddings/"
path_to_chkps = "../chkps/image_embeddings/"

# path_to_graph = os.path.join(args.path_to_graph, f"images_graph_{args.region}.pickle")
path_to_graph = os.path.join(path_to_graph, f"images_graph_{region}.pickle")
path_to_save = os.path.join(path_to_save, f"image_embeddings_{pipeline}_{region}.pickle")
# path_to_save = os.path.join(args.path_to_save, f"image_embeddings_{args.pipeline}_{args.region}.pickle")

print("starting loading graph")
#loading with torch
graph = torch.load(path_to_graph)
xs = graph.imgs[(graph.y != -1)]
ys = ys[(graph.y != -1)].tolist()
print("loaded graph successfully")

# if not os.path.exists(f"{args.path_to_save}/image_features_{region}.pickle"):
if not os.path.exists(f"{path_to_save}/image_features_{region}.pickle"):
    
    print("starting creating image features")
    image_features = get_images_features(graph)
    
    # torch.save(image_features, f"{args.path_to_save}/image_features_{region}.pickle")
    torch.save(image_features, f"{path_to_save}/image_features_{region}.pickle")
    print("image features saved")

# if args.pipeline == "basic":
if pipeline == "basic":

    full_dataset = RasterDataset(xs, ys)
else:
    
    # if args.pipeline == "efficient_delete_channels":
    if pipeline == "efficient_delete_channels":
        # throwing out some trash channels
        graph.imgs = graph.imgs[:, [0, 3, 4, 5, 6, 7, 8, 9, 10]]
    
    full_dataset = TimmRasterDataset(graph.imgs, graph.y, transforms)


feature_extractor = timm.create_model('efficientnet_b0.ra_in1k',
        pretrained=True,
        features_only=True,
        in_chans=1,
        out_indices=[2]
).to(cfg.device)

# if "efficient" in args.pipeline:
if "efficient" in pipeline:

    model = timm.create_model('efficientnet_b0.ra_in1k',
            pretrained=True,
            features_only=False,
            in_chans=12
    ).to(cfg.device)

    dataloader_all_images = torch.utils.data.DataLoader(
        full_dataset, 
        batch_size=1, 
        shuffle=False, 
        num_workers=4, 
        pin_memory=True
    )

    # use_feature_exctractor = (False if args.pipeline == "efficient_all_channels" else True)
    use_feature_exctractor = (False if pipeline == "efficient_all_channels" else True)

    model.eval()

    t = nn.AvgPool2d(kernel_size=(28, 28))

    #resulted image embeddins
    image_embeds = []

    with torch.no_grad():
        for images in tqdm(dataloader_all_images):
            if use_feature_exctractor:
                images = images.flatten(end_dim=1).unsqueeze(1)
                output = feature_extractor(images.to(cfg.device))[0]
                output = t(output).flatten(start_dim=1)
                output = torch.cat([x for x in output], dim=0)
            else:
                output = model.forward_features(images.to(cfg.device))
                output = model.forward_head(output, pre_logits=True)
            image_embeds.append(output)
            
    image_embeds = torch.cat(image_embeds, dim=0).view(len(image_embeds), -1)
    # torch.save(image_embeds.cpu(), args.path_to_save)
    torch.save(image_embeds.cpu(), path_to_save)

else:
    
    train_set, val_set = torch.utils.data.random_split(full_dataset, [0.9, 0.1])

    batch_size = 64

    dataloader_train = torch.utils.data.DataLoader(
        train_set, 
        batch_size=batch_size, 
        shuffle=True, 
    )
    dataloader_val = torch.utils.data.DataLoader(
        val_set, 
        batch_size=batch_size, 
        shuffle=False, 
    )
    
    model = RasterCNN(12).to(cfg.device)
    print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=7e-4)
    scheduler = torch.optim.lr_scheduler.CyclicLR(
        optimizer, 
        base_lr=4e-5, 
        max_lr=2e-4, 
        cycle_momentum = False, 
        step_size_up=20,
    )
    
    set_seed(111)

    best_metric = float('inf')
    best_epoch = -1

    for epoch in range(cfg.num_epochs):

        loss_i_train = []
        r2_i_train = []

        model.train()

        for images, targets in tqdm(dataloader_train, desc='Training'):

            images = images.to(cfg.device)
            targets = targets.to(cfg.device)

            optimizer.zero_grad()

            outputs = model(images.float()).squeeze()
            loss = criterion(outputs, targets.float())
            loss.backward()
            optimizer.step()

            loss_i_train.append(loss.item() ** 0.5)
            r2_i_train.append(
                r2_score(targets.cpu(), outputs.detach().cpu())
            )

        print(f"Epoch: {epoch}, Train rmse: {np.mean(loss_i_train)} r2: {np.mean(r2_i_train)}")

        loss_i_val = []
        r2_i_val = []

        model.eval()

        with torch.no_grad():
            for images, targets in dataloader_val:

                images = images.to(cfg.device)
                targets = targets.to(cfg.device)

                outputs = model(images.float()).squeeze()

                loss_i_val.append(loss.item() ** 0.5)
                r2_i_val.append(
                    r2_score(targets.cpu(), outputs.detach().cpu())
                )

        print(f"Validation rmse: [{np.mean(loss_i_val)}], r2: [{np.mean(r2_i_val)}]")

        if np.mean(loss_i_val) - best_metric < -1e6 / 2:
            best_metric = np.mean(loss_i_val)
            best_epoch = epoch

            if epoch != 0:

                # torch.save(model, f'{args.path_to_chkps}/weights_{epoch}_{round(best_metric, 5)}.pt')
                torch.save(model, f'{path_to_chkps}/weights_{epoch}_{round(best_metric, 5)}.pt')


    print('Finished Training')
    
    all_images_dataset = RasterDataset(graph.imgs, graph.y)

    dataloader_all_images = torch.utils.data.DataLoader(
        all_images_dataset, 
        batch_size=1,
        shuffle=False
    )

    hidden_states = []

    # best_model = torch.load(f'{args.path_to_chkps}/weights_{best_epoch}_{round(best_metric, 5)}.pt')
    best_model = torch.load(f'{path_to_chkps}/weights_{best_epoch}_{round(best_metric, 5)}.pt')
    
    best_model.eval()
    with torch.no_grad():
        for image, _ in tqdm(dataloader_all_images):
            _, h = best_model(image.to(cfg.device).float(), return_hidden=True)
            hidden_states.append(h.squeeze(0))
            
    hidden_states = torch.cat(hidden_states, dim=0).view(-1, hidden_states[0].shape[0])
    hidden_states = hidden_states.to(torch.device('cpu'))
    torch.save(graph, path_to_save)
5     print("Embeddings saved succesfully")

starting loading graph


FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/datasphere/s3/s3sirius/sirius_2024_participants/twwist/graph_with_cv_full_and_images/images_graph_777.pickle'

In [13]:
%run image_preprocessing.py --pipeline basic --region 812 --path_to_graph /home/jupyter/datasphere/s3/s3-sirius/sirius_2024_participants/twwist/graph_with_cv_full_and_images/

Loading graph 
Starting training basic CNN
Number of parameters: 352257


Training:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch: 0, Train rmse: 11599241.692794023 r2: -0.28612338107287677
Validation rmse: [7696757.023424346], r2: [-0.2730408361637599]


Training:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch: 1, Train rmse: 11470569.609322106 r2: -0.2733466225764176
Validation rmse: [4429126.602523798], r2: [-0.273040761028471]


...

Epoch: 98, Train rmse: 11122263.64684154 r2: -0.33203202904765944
Validation rmse: [1867609.1855332043], r2: [-0.27302822326717996]


Training:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch: 99, Train rmse: 12002266.77166435 r2: -0.2757591559166643
Validation rmse: [20995033.218517564], r2: [-0.27302888084521676]
Finished Training
Getting embeddings


  0%|          | 0/7471 [00:00<?, ?it/s]

Embeddings saved succesfully
